
# Generating Bash Code with Granite Code and Ollama


### Prerequisite: Install Ollama and Granite Code models

1. [Download and Start Ollama](https://ollama.com/download)
1. Install Granite Code 20b: `ollama pull granite-code:20b`
1. Install Granite Code 8b: `ollama pull granite-code:8b`


## One-shot Prompt with Granite Code 20b

In One-shot prompting, you provide the model with a question and no examples. The model will generate an answer given its training. Larger models tend to do better at this task.

Use the [ollama-python package](https://github.com/ollama/ollama-python) to access the model.

In [9]:
import ollama

response = ollama.generate(
  model='granite-code:20b',
  prompt="How can I use a bash script to calculate the number of days between the dates Jan 12, 2023 and Jan 20, 2024?"
)
print(response["response"])

You can use the following bash script to achieve this. 

```bash
#!/bin/bash

start_date="Jan 12, 2023"
end_date="Jan 20, 2024"

start=$(date -d "$start_date" +%s)
end=$(date -d "$end_date" +%s)

echo "Number of days between the dates = $(($((end - start))/86400))"
```

In this script, we first define the start and end dates as variables. Then, we use the `date` command with the `-d` option to convert the date strings into UNIX timestamp format using the `+%s` flag. 

Next, we subtract the start timestamp from the end timestamp and divide the result by the number of seconds in a day (86400) to get the number of days between the dates. Finally, we use the `echo` command to display the result.

Make sure to save the script with a `.sh` extension (e.g., `calculate_days.sh`), make it executable using the `chmod +x calculate_days.sh` command, and run it using `./calculate_days.sh`. The output will be the number of days between the specified dates.


## Few-shot Prompting with Granite Code 8b

In few-shot prompting, you provide the model with a question and some examples. The model will generate an answer given its training. The additional examples help the model zero in on a pattern, which may be required for smaller models to perform well at this task.

In [3]:
examples = """
Question:
Recursively finds files like '*.js', and filters out files with 'excludeddir' in path.
Answer:
find . -name '*.js' | grep -v excludeddir

Question:
Dump \"a0b\" as hexadecimal bytes
Answer:
printf \"a0b\" | od -tx1

Question:
create a tar ball of all pdf files in current folder
Answer:
find . -name *.pdf | xargs tar czvf /root/Desktop/evidence/pdf.tar

Question:
Sort all files/directories under current directory according to modification time and print only the recent 7 of them
Answer:
find -mindepth 1 -printf \"%T@ %Pn\" | sort -n -r | cut -d' ' -f 2- | tail -n +7

Question:
find all the empty directories in the current folder
Answer:
find . -type d -empty

"""

query = "How can I use a bash script to calculate the number of days between the dates Jan 12, 2023 and Jan 20, 2024?"

user_prompt = examples + "\nQuestion:\n" + query + "\nAnswer:\n"

response = ollama.chat(model='granite-code:8b', messages=[
  {
    'role': 'user',
    'content': user_prompt
  },
])
print(response['message']['content'])

#!/bin/bash
start_date=$(date -d "Jan 12, 2023" +%s)
end_date=$(date -d "Jan 20, 2024" +%s)
diff=$(( $end_date - $start_date ))
echo "$diff seconds = $(($diff/60/60/24)) days"


## Adding a System Prompt

A system prompt can be used to provide additional instructions and clarity or context for the task. Here we let the model know what we expect from it.

In [4]:
system_prompt = "You are a helpful software engineer. You write clear, concise, well-commented code."

response = ollama.chat(model='granite-code:8b', messages=[
  {
    'role':'system',
    'content': system_prompt  
  },
  {
    'role': 'user',
    'content': user_prompt
  },
])
print(response['message']['content'])


#!/bin/bash
# Set start date as Jan 12, 2023
start_date="2023-01-12"
# Set end date as Jan 20, 2024
end_date="2024-01-20"
# Calculate the number of days using a one-liner
num_days=$(date -d "$end_date" +%s -date "$start_date" +%s | bc) / (60 * 60 * 24)
echo "Number of days between $start_date and $end_date: $num_days"
